In [278]:
from jqdatasdk import *
auth('18019156072','156072')
import pandas as pd
import numpy as np
import scipy.stats as st
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [279]:
def quantile_combination_test_Head_Return(factors_data, close_data, trade_days, stock_index=''):
    """
    对列表中所有因子进行分位数组合测试, factors_data是Multiindex DataFrame, close_data是每N个交易日的收盘价,
    记第一个有因子数据的周期为0, 一直到周期M, 则trade_days从周期0最后一个交易日开始，到周期M+1最后一个交易日
    如果股票池为指数, stock_index赋值为000300.XSHG
    """
 
    # 获取因子名列表
    factors_set = np.unique(factors_data.index.get_level_values(0))
    # 创建空DataFrame用于储存多空组合收益R1-R5 
    Head_Return = pd.DataFrame(None, index=trade_days, columns=factors_set)
    # 获取trade_days[0]这一交易日在factors_data和close_data中对应的序号
    factors_data_t0 = np.searchsorted(factors_data.columns, trade_days[0])
    close_data_t0 = np.searchsorted(close_data.columns, trade_days[0])

    for j in range(len(factors_set)):
        factor = factors_set[j]
        # 获取单个因子的子DataFrame
        factor_data = factors_data.loc[factor]
        for i in range(len(trade_days)//4):
            # 计算本期和下期共有的股票
            
            common_stocks = list(factor_data.iloc[:, (factors_data_t0 + 20 * i - 4): (factors_data_t0 + 20 * i + 36)].dropna().index)
            common_stocks = list(close_data.loc[common_stocks, :].iloc[:, (close_data_t0 + 4 * i + 3): (close_data_t0  + 4 * i + 8)].dropna().index)


            # 计算本期因子排序前20%和后20%的股票
            factor_exposures = factor_data.loc[common_stocks, :].iloc[:, (factors_data_t0 + 20 * i - 4): (factors_data_t0 + 20 * i + 16)].mean(1).reindex(common_stocks)
            factor_exposures = factor_exposures.to_frame()
            factor_exposures = factor_exposures.sort_values(by=[0],ascending = False)
            tail = factor_exposures.tail(60).index
            # 计算Head收益率
            head_d0 = close_data.loc[head].iloc[:, 4 * i + 3]
            head_d1 = close_data.loc[head].iloc[:, 4 * i + 7]
            head_returns = ((head_d1 - head_d0)/head_d0).reindex(head)
            head_return = np.sum(head_returns)/60
            Head_Return[factor][i-1] = head_return            
        print('Head分位数测试计算进度{}/{}'.format(j+1, len(factors_set)))
    return Head_Return

def quantile_combination_test_Tail_Return(factors_data, close_data, trade_days, stock_index=''):
    """
    对列表中所有因子进行分位数组合测试, factors_data是Multiindex DataFrame, close_data是每N个交易日的收盘价,
    记第一个有因子数据的周期为0, 一直到周期M, 则trade_days从周期0最后一个交易日开始，到周期M+1最后一个交易日
    股票池为指数, stock_index赋值为000300.XSHG
    """
 
    # 获取因子名列表
    factors_set = np.unique(factors_data.index.get_level_values(0))
    # 创建空DataFrame用于储存多空组合收益R1-R5 
    Tail_Return = pd.DataFrame(None, index=trade_days, columns=factors_set)
    # 获取trade_days[0]这一交易日在factors_data和close_data中对应的序号
    factors_data_t0 = np.searchsorted(factors_data.columns, trade_days[0])
    close_data_t0 = np.searchsorted(close_data.columns, trade_days[0])

    for j in range(len(factors_set)):
        factor = factors_set[j]
        # 获取单个因子的子DataFrame
        factor_data = factors_data.loc[factor]
        for i in range(len(trade_days)//4):
            # 计算本期和下期共有的股票
            common_stocks = list(factor_data.iloc[:, (factors_data_t0 + 20 * i - 4): (factors_data_t0 + 20 * i + 36)].dropna().index)
            common_stocks = list(close_data.loc[common_stocks, :].iloc[:, (close_data_t0 + 4 * i + 3): (close_data_t0  + 4 * i + 8)].dropna().index)

            
            # 计算本期因子排序前20%和后20%的股票
            #factor_exposures = factor_data.loc[common_stocks, :].iloc[:, (factors_data_t0 + 5 * i - 4): (factors_data_t0 + 5 * i + 1)].mean(1).reindex(common_stocks)
            factor_exposures = factor_data.loc[common_stocks, :].iloc[:, (factors_data_t0 + 20 * i - 4): (factors_data_t0 + 20 * i + 16)].mean(1).reindex(common_stocks)

            
            factor_exposures = factor_exposures.to_frame()
            factor_exposures = factor_exposures.sort_values(by=[0],ascending = False)
            tail = factor_exposures.tail(60).index
            # 获取权重列表
            #weights = get_index_weights(stock_index, date=trade_days[i]).reindex(common_stocks).weight
            #计算Head收益率           
            tail_d0 = close_data.loc[tail].iloc[:, 4 * i + 3]
            tail_d1 = close_data.loc[tail].iloc[:, 4 * i + 7]
            tail_returns = ((tail_d1 - tail_d0)/tail_d0).reindex(tail)
            tail_return = np.sum(tail_returns)/60
            Tail_Return[factor][i-1] = tail_return
            
        print('Tail分位数测试计算进度{}/{}'.format(j+1, len(factors_set)))
    return Tail_Return


In [280]:
factors_data = pd.read_csv(r'/Users/kitty/Desktop/quant_project/with_tech_fact_all_data.csv',index_col=0)
factors_data = factors_data.set_index([factors_data.index,'code'])
close_data = pd.read_csv(r'/Users/kitty/Desktop/quant_project/close_data_2014_07_31.csv',index_col=0)
trade_days = list(np.intersect1d(factors_data.columns, close_data.columns[: -1]))
Head_Return = quantile_combination_test_Head_Return(factors_data, close_data, trade_days, '000300.XSHG')
Tail_Return = quantile_combination_test_Tail_Return(factors_data, close_data, trade_days, '000300.XSHG')





Head分位数测试计算进度1/46
Head分位数测试计算进度2/46
Head分位数测试计算进度3/46
Head分位数测试计算进度4/46
Head分位数测试计算进度5/46
Head分位数测试计算进度6/46
Head分位数测试计算进度7/46
Head分位数测试计算进度8/46
Head分位数测试计算进度9/46
Head分位数测试计算进度10/46
Head分位数测试计算进度11/46
Head分位数测试计算进度12/46
Head分位数测试计算进度13/46
Head分位数测试计算进度14/46
Head分位数测试计算进度15/46
Head分位数测试计算进度16/46
Head分位数测试计算进度17/46
Head分位数测试计算进度18/46
Head分位数测试计算进度19/46
Head分位数测试计算进度20/46
Head分位数测试计算进度21/46
Head分位数测试计算进度22/46
Head分位数测试计算进度23/46
Head分位数测试计算进度24/46
Head分位数测试计算进度25/46
Head分位数测试计算进度26/46
Head分位数测试计算进度27/46
Head分位数测试计算进度28/46
Head分位数测试计算进度29/46
Head分位数测试计算进度30/46
Head分位数测试计算进度31/46
Head分位数测试计算进度32/46
Head分位数测试计算进度33/46
Head分位数测试计算进度34/46
Head分位数测试计算进度35/46
Head分位数测试计算进度36/46
Head分位数测试计算进度37/46
Head分位数测试计算进度38/46
Head分位数测试计算进度39/46
Head分位数测试计算进度40/46
Head分位数测试计算进度41/46
Head分位数测试计算进度42/46
Head分位数测试计算进度43/46
Head分位数测试计算进度44/46
Head分位数测试计算进度45/46
Head分位数测试计算进度46/46
Tail分位数测试计算进度1/46
Tail分位数测试计算进度2/46
Tail分位数测试计算进度3/46
Tail分位数测试计算进度4/46
Tail分位数测试计算进度5/46
Tail分位数测试计算进度6/46
Tail分位数测试计算进度7/46
Tail分位数测试

In [281]:
Head_Return = Head_Return+1
Head_Return = Head_Return.fillna(1)
factors_set = np.unique(factors_data.index.get_level_values(0))
Head_Return_multiplication = pd.DataFrame(None, index=['head_return'], columns=factors_set)
for j in range(len(factors_set)):
        head_return = 1
        factor = factors_set[j]
        for i in range(60):
            head_return = head_return*Head_Return[factor][i]
        Head_Return_multiplication[factor] = head_return



In [282]:
Tail_Return = Tail_Return + 1
Tail_Return = Tail_Return.fillna(1)
factors_set = np.unique(factors_data.index.get_level_values(0))
Tail_Return_multiplication = pd.DataFrame(None, index=['tail_return'], columns=factors_set)

for j in range(len(factors_set)):
        tail_return = 1
        factor = factors_set[j]
        for i in range(60):
            tail_return = tail_return*Tail_Return[factor][i]
        
        Tail_Return_multiplication[factor] = tail_return

In [283]:
return_head_and_tail = pd.concat([Head_Return_multiplication, Tail_Return_multiplication])
return_head_and_tail = return_head_and_tail.append(pd.Series(name='difference'))
for j in range(len(return_head_and_tail.columns)):        
        factor = return_head_and_tail.columns[j]
        return_head_and_tail[factor][2] = return_head_and_tail[factor][0]-return_head_and_tail[factor][1]
return_head_and_tail.to_csv('Monthly_Head_Tail_difference_with_tech_factor.csv')